In [1]:
#https://towardsdatascience.com/text-classification-with-pytorch-7111dae111a6

In [30]:
import torch
from torch.nn import functional as F
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.data.utils import ngrams_iterator
from torch.autograd import Variable
import time
import copy
from torch.optim import lr_scheduler
from torch.utils.data.dataset import random_split
from torchtext import data
from torch.utils.data import DataLoader

from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe
import spacy
spacy.load('en_core_web_sm')
spacy.prefer_gpu()
from wordcloud import WordCloud, STOPWORDS
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
import os

from sklearn.model_selection import train_test_split


train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
submit_sample_data = pd.read_csv('sample_submission.csv')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/dawidkubicki/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## Plan of attack

1. Preprocess data
    a) Delete unused columns
    b) Clean data - as it done in tensorflow notebook
    c) Split data with train_test_split from scikit
    d) Prepare tokens with tf tokenizer or pytorch thing
    e) Pad sequence or Pytorch thing
    
2. Prepare the model
    a) simple LSTM
    b) Bidirectional LSTM
    c) BERT
    
3. Prepare function to get batches

4. Write training phase

5. Write evaluation

6. Submit it

In [4]:
#preprocessing the data
X_train = train_data.drop(columns=["id", "keyword", "location"])
X_test = test_data.drop(columns=["id", "keyword", "location"])

In [5]:
X_train.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', text)

lemmatizer = WordNetLemmatizer()

X_train["text"] = X_train["text"].apply(lambda x: clean_text(x))
X_test["text"] = X_test["text"].apply(lambda x: clean_text(x))

X_train["text"] = X_train["text"].apply(lambda x: remove_emoji(x))
X_test["text"] = X_test["text"].apply(lambda x: remove_emoji(x))

X_train["text"] = X_train["text"].apply(lambda x: lemmatizer.lemmatize(x))
X_test["text"] = X_test["text"].apply(lambda x: lemmatizer.lemmatize(x))

In [7]:
df_train = X_train['text']
df_test = X_test['text']

punctuations = string.punctuation
nlp_spacy = spacy.load('en_core_web_sm')
stop_words = STOP_WORDS
parser = English()
stopwords = set(STOPWORDS)

def spacy_preprocessor(sentence):
    result = nlp_spacy(sentence)
    return result

df_train = df_train.apply(lambda x: spacy_preprocessor(x))
df_test = df_test.apply(lambda x: spacy_preprocessor(x))

In [8]:
X_train['text'] = df_train
X_test['text'] = df_test

In [9]:
X = X_train[[col for col in X_train.columns if not col == 'target']]
y = X_train['target']
X_Train, X_Valid, y_Train, y_Valid = train_test_split(X, y, test_size=0.2, random_state=20)

train_data = pd.concat([X_Train, y_Train], axis=1)
valid_data = pd.concat([X_Valid, y_Valid], axis=1)

# save them
!mkdir preprocessed_data
train_data.to_csv('preprocessed_data/train.csv', index=False)
valid_data.to_csv('preprocessed_data/valid.csv', index=False)
X_test.to_csv('preprocessed_data/test.csv', index=False)

mkdir: cannot create directory ‘preprocessed_data’: File exists


## Fun with dataset and model

In [10]:
is_cuda = torch.cuda.is_available()
print(f"CUDA status is {is_cuda}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CUDA status is True


#### Let's process the string with a fixed length

In [16]:
fix_length = 20 
text = data.Field(tokenize=spacy.load('en_core_web_sm'),
                 pad_first=True)

text.build_vocab(train_data, valid_data)

In [17]:
VOCAB_SIZE = len(text.vocab)
NGRAMS = 2
BATCH_SIZE = 8
EMBED_DIM = 32
NUM_CLASS = 2

In [24]:
#class of model
class TwitterNet(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()
        
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)
    
model = TwitterNet(VOCAB_SIZE, EMBED_DIM, NUM_CLASS).to(device)

In [25]:
def generate_batch(batch):
    label = torch.tensor([int(entry.target[0]) for entry in batch])
    _text = []
    for entry in batch:
        _entry = []
        for t in entry.text:
            _entry.append(text.vocab.stoi[t])
        _text.append(torch.tensor(_entry,dtype=torch.long))
    offsets = [0] + [len(entry) for entry in _text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    _text = torch.cat(_text)
    return _text, offsets, label

In [26]:
def train_func(sub_train_):
    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()
    # Adjust the learning rate
    scheduler.step()
    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()
    return loss / len(data_), acc / len(data_)

In [31]:
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_data) * 0.95)
sub_train_, sub_valid_ = random_split(train_data, [train_len, len(train_data) - train_len])

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)
    
    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60
    
    print(f'Epoch: {epoch + 1}, | time in {mins} minutes and {secs} seconds')
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

KeyError: 5742